In [1]:
from __future__ import print_function
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StandardScaler
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [2]:
if __name__ == "__main__":
    spark = SparkSession\
        .builder\
        .appName("Practice")\
        .getOrCreate()

In [3]:
spark.version   

'3.0.2'

In [4]:
df = spark.read.csv('Admission_Prediction.csv',header=True)
df.show()

+---------+-----------+-----------------+----+----+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating| SOP| LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+----+----+----+--------+---------------+
|   337.00|     118.00|                4|4.50|4.50|9.65|    1.00|           0.92|
|   324.00|     107.00|                4|4.00|4.50|8.87|    1.00|           0.76|
|     null|     104.00|                3|3.00|3.50|8.00|    1.00|           0.72|
|   322.00|     110.00|                3|3.50|2.50|8.67|    1.00|           0.80|
|   314.00|     103.00|                2|2.00|3.00|8.21|    0.00|           0.65|
|   330.00|     115.00|                5|4.50|3.00|9.34|    1.00|           0.90|
|   321.00|     109.00|             null|3.00|4.00|8.20|    1.00|           0.75|
|   308.00|     101.00|                2|3.00|4.00|7.90|    0.00|           0.68|
|   302.00|     102.00|                1|2.00|1.50|8.00|    0.00|           0.50|
|   323.00|     

In [5]:
type(df)

pyspark.sql.dataframe.DataFrame

In [6]:
from pyspark.sql.functions import col, count, isnan, when

In [7]:
# show 10 entries of GRE Score column
df.select("GRE Score").show(10)  # sql commands

+---------+
|GRE Score|
+---------+
|   337.00|
|   324.00|
|     null|
|   322.00|
|   314.00|
|   330.00|
|   321.00|
|   308.00|
|   302.00|
|   323.00|
+---------+
only showing top 10 rows



In [8]:
df.select("TOEFL Score","University Rating", "SOP", "LOR","CGPA").show(10)

+-----------+-----------------+----+----+----+
|TOEFL Score|University Rating| SOP| LOR|CGPA|
+-----------+-----------------+----+----+----+
|     118.00|                4|4.50|4.50|9.65|
|     107.00|                4|4.00|4.50|8.87|
|     104.00|                3|3.00|3.50|8.00|
|     110.00|                3|3.50|2.50|8.67|
|     103.00|                2|2.00|3.00|8.21|
|     115.00|                5|4.50|3.00|9.34|
|     109.00|             null|3.00|4.00|8.20|
|     101.00|                2|3.00|4.00|7.90|
|     102.00|                1|2.00|1.50|8.00|
|     108.00|                3|3.50|3.00|8.60|
+-----------+-----------------+----+----+----+
only showing top 10 rows



In [9]:
# Show SOP and assign 0 or 1 depending on SOP
df.select("SOP",when(df.SOP<= 3, 
0).otherwise(1)).show(10)

+----+--------------------------------------+
| SOP|CASE WHEN (SOP <= 3) THEN 0 ELSE 1 END|
+----+--------------------------------------+
|4.50|                                     1|
|4.00|                                     1|
|3.00|                                     0|
|3.50|                                     0|
|2.00|                                     0|
|4.50|                                     1|
|3.00|                                     0|
|3.00|                                     0|
|2.00|                                     0|
|3.50|                                     0|
+----+--------------------------------------+
only showing top 10 rows



In [10]:
# Show rows with specified authors if in the given options
df [df.CGPA.isin("9.65")].show(100)

+---------+-----------+-----------------+----+----+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating| SOP| LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+----+----+----+--------+---------------+
|   337.00|     118.00|                4|4.50|4.50|9.65|    1.00|           0.92|
+---------+-----------+-----------------+----+----+----+--------+---------------+



In [11]:
df.select("CGPA",when(df.CGPA>=9,"Extraordinary").otherwise("Good marks")).show(10)

+----+------------------------------------------------------------+
|CGPA|CASE WHEN (CGPA >= 9) THEN Extraordinary ELSE Good marks END|
+----+------------------------------------------------------------+
|9.65|                                               Extraordinary|
|8.87|                                                  Good marks|
|8.00|                                                  Good marks|
|8.67|                                                  Good marks|
|8.21|                                                  Good marks|
|9.34|                                               Extraordinary|
|8.20|                                                  Good marks|
|7.90|                                                  Good marks|
|8.00|                                                  Good marks|
|8.60|                                                  Good marks|
+----+------------------------------------------------------------+
only showing top 10 rows



In the brackets of “Like” function, % character is used to filter out all LOR having “ 4 ” word. If the condition we are looking for is the exact match, then no % character shall be used.

In [12]:
# Show LOR is TRUE if LOR has 4.00
df.select("LOR",
df.LOR.like("%4.00%")).show(15)

+----+---------------+
| LOR|LOR LIKE %4.00%|
+----+---------------+
|4.50|          false|
|4.50|          false|
|3.50|          false|
|2.50|          false|
|3.00|          false|
|3.00|          false|
|4.00|           true|
|4.00|           true|
|1.50|          false|
|3.00|          false|
|4.00|           true|
|4.50|          false|
|4.50|          false|
|3.00|          false|
|2.00|          false|
+----+---------------+
only showing top 15 rows



StartsWith scans from the beginning of word/content with specified criteria in the brackets. In parallel, EndsWith processes the word/content starting from the end. Both of the functions are case sensitive.

In [13]:
df.select("LOR", df.LOR.startswith("4")).show(5)

+----+------------------+
| LOR|startswith(LOR, 4)|
+----+------------------+
|4.50|              true|
|4.50|              true|
|3.50|             false|
|2.50|             false|
|3.00|             false|
+----+------------------+
only showing top 5 rows



In [14]:
df.select("LOR", df.LOR.endswith(".00")).show(5)

+----+------------------+
| LOR|endswith(LOR, .00)|
+----+------------------+
|4.50|             false|
|4.50|             false|
|3.50|             false|
|2.50|             false|
|3.00|              true|
+----+------------------+
only showing top 5 rows



In [15]:
import pyspark.sql.functions as F
df= df.withColumn("LOR2", 2*F.col("LOR")) # create new column with 2*LOR

In [16]:
df.show()

+---------+-----------+-----------------+----+----+----+--------+---------------+----+
|GRE Score|TOEFL Score|University Rating| SOP| LOR|CGPA|Research|Chance of Admit|LOR2|
+---------+-----------+-----------------+----+----+----+--------+---------------+----+
|   337.00|     118.00|                4|4.50|4.50|9.65|    1.00|           0.92| 9.0|
|   324.00|     107.00|                4|4.00|4.50|8.87|    1.00|           0.76| 9.0|
|     null|     104.00|                3|3.00|3.50|8.00|    1.00|           0.72| 7.0|
|   322.00|     110.00|                3|3.50|2.50|8.67|    1.00|           0.80| 5.0|
|   314.00|     103.00|                2|2.00|3.00|8.21|    0.00|           0.65| 6.0|
|   330.00|     115.00|                5|4.50|3.00|9.34|    1.00|           0.90| 6.0|
|   321.00|     109.00|             null|3.00|4.00|8.20|    1.00|           0.75| 8.0|
|   308.00|     101.00|                2|3.00|4.00|7.90|    0.00|           0.68| 8.0|
|   302.00|     102.00|                1|2.

In [17]:
df = df.drop("LOR2") # to drop the column

In [18]:
df.show()

+---------+-----------+-----------------+----+----+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating| SOP| LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+----+----+----+--------+---------------+
|   337.00|     118.00|                4|4.50|4.50|9.65|    1.00|           0.92|
|   324.00|     107.00|                4|4.00|4.50|8.87|    1.00|           0.76|
|     null|     104.00|                3|3.00|3.50|8.00|    1.00|           0.72|
|   322.00|     110.00|                3|3.50|2.50|8.67|    1.00|           0.80|
|   314.00|     103.00|                2|2.00|3.00|8.21|    0.00|           0.65|
|   330.00|     115.00|                5|4.50|3.00|9.34|    1.00|           0.90|
|   321.00|     109.00|             null|3.00|4.00|8.20|    1.00|           0.75|
|   308.00|     101.00|                2|3.00|4.00|7.90|    0.00|           0.68|
|   302.00|     102.00|                1|2.00|1.50|8.00|    0.00|           0.50|
|   323.00|     

In [19]:
df.printSchema()

root
 |-- GRE Score: string (nullable = true)
 |-- TOEFL Score: string (nullable = true)
 |-- University Rating: string (nullable = true)
 |-- SOP: string (nullable = true)
 |-- LOR: string (nullable = true)
 |-- CGPA: string (nullable = true)
 |-- Research: string (nullable = true)
 |-- Chance of Admit: string (nullable = true)



In [20]:
from pyspark.sql.functions import col
df = df.select(*(col(c).cast("float").alias(c) for c in df.columns)) # convert to float 

In [21]:
df.printSchema()

root
 |-- GRE Score: float (nullable = true)
 |-- TOEFL Score: float (nullable = true)
 |-- University Rating: float (nullable = true)
 |-- SOP: float (nullable = true)
 |-- LOR: float (nullable = true)
 |-- CGPA: float (nullable = true)
 |-- Research: float (nullable = true)
 |-- Chance of Admit: float (nullable = true)



In [22]:
#checking for null ir nan type values in our columns
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|       15|         10|               15|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [23]:
df = df.fillna(0) # replace it with zero
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|        0|          0|                0|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+

